# Regression - Audio

## Imports

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../')

from tcn import TCN
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from data_preparation import prepare_x_data, get_Y_labels, unscale_Y, reshape_Y, reshape_X

## Data preparation

Data preparation includes: <br>
<ul>
  <li>selecting necessary features from source files</li>
  <li>creating combined dataset for the model training</li>
  <li>reshaping data for model training.</li>

In [ ]:
scaler = StandardScaler()

In [ ]:
x_train = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler)

y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train),1,1)

In [ ]:
x_test = prepare_x_data('../Data/LLDs_audio_eGeMAPS/dev',';', 2, scaler) 

y_test = get_Y_labels('../Data/labels_metadata.csv', 0, 60, scaler)
y_test = reshape_Y(y_test,len(x_test),1,1)

In [ ]:
# Get the longest sequence length.
len_dict = {}


for idx, df in enumerate(x_train):
    len_dict.update({idx:len(df)})

max_len = max(len_dict, key=len_dict.get)

max_len = {max_len:len_dict.get(max_len)}

print(f'Longest sequence {max_len} ')

# Temporal Convolutional Neural Network

Proposed model for temporal convolutional neural network architecture for multi-modal dataset. Both data modalities are concatenated together before the last Dense layer.

In [ ]:
# Define model.
model = Sequential()
model.add(TCN(nb_filters=12, kernel_size=33, dilations=(256,512,1024,2048), padding='causal', nb_stacks=1,
                use_batch_norm=False, use_layer_norm=False, use_weight_norm=False, use_skip_connections=True, dropout_rate=0.005, return_sequences=False, input_shape=(None, 23)))
model.add(Dense(1, activation='linear'))

# Define optimizer and show summary.
model.compile(optimizer=Adam(learning_rate=0.004), loss='mse', metrics=['mae'])
early_stop = EarlyStopping(monitor='loss', patience=10)
model.summary()

In [ ]:
plot_model(model=model, show_dtype=False, show_layer_names=True, show_shapes=True, to_file='TCN_audio.png')

### Model training

Model training setup is based on an iterative approach where model is trained one file at a time, then learned parameters are saved and loaded in the next iterative step. This setup is necessary due to the fact that source files does not have an uniform size and differ in number of frames.

In [ ]:
# Fit the model one file at a time.
train_index = 0
val_index = 0
loss = {}
train_eval = []
train_loss = []
train_mae = []

for train in x_train:

    train = np.array(train).reshape((1, train.shape[0], -1))
    history = model.fit(x=train, y=y_train[train_index], epochs=10, shuffle=False, batch_size=32, verbose=0, callbacks=early_stop)
    loss.update({train_index:history.history})

    train_index += 1
    
    model.save(r'TCN_audio', include_optimizer=True) # Save model configuration to Saved_models.
    model = load_model(r'TCN_audio', custom_objects={'TCN':TCN}) # Load model configuration from Saved_models.

    scores = model.evaluate([train], y_train[val_index], verbose = 0)
    train_eval.append(scores)
    train_loss.append(scores[0])
    train_mae.append(scores[1])

    val_index += 1


In [ ]:
avg_train_loss = sum(train_loss) / len(train_loss)
avg_train_mae = sum(train_mae) / len(train_mae)
print("Train loss visual (avg):", avg_train_loss, "Train MAE visual (avg):", avg_train_mae)

### Model evaluation

Model evaluation is performed on subset taken from test data. 

In [ ]:
x_val = x_test[:30].copy()
y_val = y_test[:30].copy()

In [ ]:
eval_loss = []
eval_mae = []
eval_visual = []
index = 0

for input in x_val:
    input = reshape_X(input)
    scores = model.evaluate(input, y_val[index], verbose = 0)
    
    eval_visual.append(scores)
    eval_loss.append(scores[0])
    eval_mae.append(scores[1])
    
    index += 1

In [ ]:
avg_eval_loss = sum(eval_loss) / len(eval_loss)
avg_eval_mae = sum(eval_mae) / len(eval_mae)
print("Validation loss visual (avg):", avg_eval_loss, "Validation MAE visual (avg):", avg_eval_mae)

In [ ]:
plt.xlabel("Iterations")
plt.ylabel("Scaled YMRS value")
plt.plot(eval_loss, label="MSE")
plt.plot(eval_mae, label="MAE")
plt.legend()
plt.show()

### YMRS prediction and comparison

Prediction is made on different subset taken from the test dataset. Then actual and predicted YMRS values are compared.

In [ ]:
x_test  = x_test[30:].copy()
y_actual = y_test[30:].copy()

In [ ]:
prediction = []
pred_eval = []
pred_loss = []
pred_mae = []
test_index = 0

for file in x_test:
    file = np.array(file).reshape(1, file.shape[0], -1)

    pred = model.predict(file)

    prediction.append(pred)
    scores = model.evaluate(file, y_actual[test_index], verbose=0)
    pred_eval.append(scores)
    pred_loss.append(scores[0])
    pred_mae.append(scores[1])


    test_index += 1

In [ ]:
avg_pred_loss = sum(pred_loss) / len(pred_loss)
avg_pred_mae = sum(pred_mae) / len(pred_mae)
print("Prediction loss visual (avg):", avg_pred_loss, "Prediction MAE visual (avg):", avg_pred_mae)

In [ ]:
prediction = np.array(prediction).reshape(-1, 1)
y_actual = y_actual.reshape(-1, 1)

prediction = unscale_Y(prediction,scaler)
y_actual = unscale_Y(y_actual,scaler)

In [ ]:
pred_df = pd.DataFrame(data=np.column_stack((y_actual, prediction)), columns=['y_actual','y_pred'])
pred_df['pred_error'] = pred_df['y_actual'] - pred_df['y_pred']
pred_df = pred_df.sort_values(by=['y_actual']).reset_index()
pred_df['y_actual'] = pred_df['y_actual'].apply(np.int64)
pred_df

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mse = mean_squared_error(pred_df.y_actual, pred_df.y_pred)
mae = mean_absolute_error(pred_df.y_actual, pred_df.y_pred)

mse, mae

In [ ]:
# Scatter plot x = actual, y = predicted
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.scatter(pred_df.y_actual, pred_df.y_pred)
plt.show()

In [ ]:
# Scatter plot x = actual, y = actual - predicted
plt.xlabel('Actual')
plt.ylabel('Prediction error')
plt.scatter(pred_df.y_actual, pred_df.pred_error)
plt.show()

In [ ]:
plt.xlabel("Test subject IDs")
plt.ylabel("Target value (YMRS)")
plt.plot(pred_df.y_actual, label="Actual")
plt.plot(pred_df.y_pred, label="Predicted")
plt.legend()
plt.show()